In [3]:
import random
import numpy as np

In [22]:
class TheGame:

    def __init__(self):
        '''
        Initialising function for TheGame class object
        '''
        self.drawing_pile = [i for i in range(2,100)] # drawing pile
        random.shuffle(self.drawing_pile) # it is shuffled

        self.hand = self.drawing_pile[:8] # Take the first 8 card
        self.hand.sort() # sorting is regularly done for iteration purposes
        self.drawing_pile = self.drawing_pile[8:] # Card that are drawn cannot remain in drawing pile

        # Stacks on which to play cards inhand
        self.stacks = {"up1": [1], # Two stacks may only rise
                       "up2": [1], # (except for situations where a card is exactly 10 below the current one)
                       "down1": [100], # Two stacks may only fall
                       "down2": [100]} # (except for situations where a card is exactly 10 above th current one)
        
        # End-of-game messages to output in case of game over or win.
        self.game_over = rf"No move possible anymore."
        self.complete = rf"Congratulations. You have beat the game."

    def recognizing_ten_diffs(self,hand):
        '''
        Function that creates list items within hand for cards that must be played alongside each other,
        because of exact 10-difference between them.
        Input:
            - Hand (function requires hard copy of self.hand, to not destroy original item just yest)
        Output:
            - Hand (correctly overwritten hand with new list elements)
        '''
        tenners = [] # Empty list of ten-diffs

        # Perform double iteration
        for nr in hand:
            if type(nr) == int: # Comparing individual cards not belonging ot any ten-diff yet
                for nr2 in hand:
                    if type(nr) == int:
                        if abs(nr - nr2) == 10:
                            diff_item = [nr, nr2] # Elements are compounded in a list
                            diff_item.sort()
                            tenners.append(diff_item)
                    else:
                        for item in nr2: # Comparing individual cards with existing ten-diff list items
                            if abs(item - nr) == 10:
                                diff_item = nr2.append(nr)
                                diff_item.sort() 
                                tenners.append(diff_item)
                            
        # Removing double ten-diffs stemming from double iteration in line 38
        ten_diffs = np.unique(np.array([np.sort(sub) for sub in tenners]), axis=0).tolist()

        # If a set of ten-diffs can be compounded into one "move", they must
        ten_diffs_copy = ten_diffs.copy() # Set hard copy for double iteration
        for item in ten_diffs:
            for item2 in ten_diffs:
                if item[-1] == item2[0]: # Ten-diffs will have linked beginning and end numbers
                    new_item = list(set(item + item2))
                    new_item.sort()
                    ten_diffs_copy.append(new_item)
                    ten_diffs_copy.remove(item)
                    ten_diffs_copy.remove(item2)

        # Adding full set of ten-diffs as list items to hand 
        for item in ten_diffs_copy:
            hand.append(item)
            for elem in item:
                hand.remove(elem)

        return hand
    
    def ten_moves(self):
        '''
        Function that calls recognizing_ten_diffs to alter hand,
        and then see if cards fill between the ten-diff's range, so a "free" card can be played
        Output:
            - Altered hand
        '''
        self.hand = self.recognizing_ten_diffs(self.hand)

        for nr in reversed(self.hand):
            if type(nr) == int:
                for tenner in self.hand:
                    if type(tenner) == list:
                        if tenner[0] < nr < tenner[-1]: # A free card fits between two combo move cards
                            tenner.append(nr)
                            self.hand.remove(nr)
                
        return self.hand
    
    def optimal_move(matrix):
        '''
        Function that finds the coordinates of the minimal value in a 2D-matrix
        Output:
            - 2D-array argmin
        '''
        return list(zip(*np.where(matrix == matrix.min())))
    
    def analyze_move(self):
        '''
        Function that analyses all possible moves one can play with their hand
        '''

        # Initialise a matrix of size 4 (for each stack) by hand length of infinity's.
        playing_matrix = np.matrix(np.ones((4, len(self.hand))) * np.inf) # In case a card cannot be played, infinity is left unchanged.

        elem_counter = 0 # Counter that keeps track of x-coordinates in matrix

        # Iterate over all cards
        for elem in self.hand:
            stack_counter = 0 # Counter that keeps track of y-coordinates in matrix

            # If card is not part of a combo move
            if type(elem) == int: 

                # Iterate over all stacks
                for key, val in self.stacks.items(): # val represents the current stack before move is made
                    if key.startswith("up"): 

                        # The first two stacks may only rise
                        if elem > val[0]: 
                            playing_matrix[stack_counter, elem_counter] = elem - val[0] # compute damage to stack
                        
                        # Except for cards that have an exact minus-10 diff
                        elif val[0] - elem == 10: 
                            playing_matrix[stack_counter, elem_counter] == -10
                    
                    # The second two stacks may only fall
                    else:
                        if elem < val[0]:
                            playing_matrix[stack_counter, elem_counter] = val[0] - elem
                        elif val[0] - elem == (-10):
                            playing_matrix[stack_counter, elem_counter] == -10

                    stack_counter += 1

            # If card is part of a combo move
            else:
                for key, val in self.stacks.items():
                    if key.startswith("up"):

                        # Compute distance to final card in combo move
                        if max(elem) > val[0]: # The highest card in the combo must still be above the current stack card
                            playing_matrix[stack_counter, elem_counter] = min(elem) - val[0]
                        elif val[0] - max(elem) == 10:
                            playing_matrix[stack_counter, elem_counter] == -10
                    else:
                        if min(elem) < val[0]: # The lowest card in the combo must still be below the current stack card
                            playing_matrix[stack_counter, elem_counter] = val[0] - max(elem)
                        elif val[0] - min(elem) == (-10):
                            playing_matrix[stack_counter, elem_counter] == -10

                    stack_counter += 1

            elem_counter += 1

        return playing_matrix
    
    def play_move(self):
        '''
        Function that takes the best-move analysis and plays said move.
        Returns:
            - card_counter (the amount of cards that have contributed to playing the move)
            - print statement to keep track of new status of hand and stacks.
        '''
        # The game is finished when the hand and drawing pile are all out
        if len(self.hand) == 0 and len(self.drawing_pile) == 0: 
            return self.complete
        
        matrix = self.analyze_move() # analyse best move
        print(matrix.min())
        best_move = self.optimal_move(matrix) # find matrix coordinates of best move
        stacklist = list(game_instance.stacks.keys()) # iterable list item for alteration

        # If no other move can be played (only inf's remain), the game is lost.
        if matrix.min() == np.inf: 
            return self.game_over

        # If there is a tie in best moves...
        elif len(best_move) != 1:

            # Initialising lists to compute tiebreaker statistics
            tied_best_cards, stacks_for_tied_best_cards, dist_from_base = [], [], [] 

            # Inspect which cards correspond to tied best moves
            for move in best_move:
                stack, card = list(move)[0], list(move)[1]
                tied_best_cards.append(self.hand[card])
                stacks_for_tied_best_cards.append(stack)

                # Compute the distance of the card to the base of its ideal stack (1 or 100)
                if stack in [0,1]:
                    dist_from_base.append(card-1)
                else:
                    dist_from_base.append(100-card)

                # Ultimately choose for the one with minimal distance from base
                choice = dist_from_base.index(min(dist_from_base)) # in case the tie is still not broken, .index() picks to first option
            
            best_move = best_move[choice]
        
        # Set stack and card variables to the corresponding right items in self.hand and self.stacks
        try:
            stack, card = list(best_move)[0], list(best_move)[1]
        except: # Try lines for debugging purposes, second run breaks down otherwise
            stack, card = list(best_move)[0][0], list(best_move)[0][1]
        print(rf"Best card to play: {self.hand[card]} on stack {stack + 1}.")

        # Keep track of the amount of cards that is played with a (combo) move
        card_counter = 0

        # In case the best card is not part of a combo move
        if type(self.hand[card]) == int: 
            self.stacks[stacklist[stack]].insert(0, self.hand[card]) # perform the move
            self.hand.remove(self.hand[card]) # remove card from hand
            card_counter += 1

        # In case the best card is part of a combo move
        else:
            if stack in [0,1]:

                # First play the "free cards" as defined in in line 86
                for elem in self.hand[card]:
                    if (elem - self.hand[card][0]) % 10 != 0: # Way to distinguish free cards from combo move cards
                        self.stacks[stacklist[stack]].insert(0, elem)
                        card_counter += 1
                for elem in reversed(self.hand[card]):
                    if (elem - self.hand[card][0]) % 10 == 0:
                        self.stacks[stacklist[stack]].insert(0, elem)
                        card_counter += 1

            else:
                for elem in self.hand[card]:
                    if (elem - self.hand[card][0]) % 10 != 0:
                        self.stacks[stacklist[stack]].insert(0, elem)
                        card_counter += 1
                for elem in self.hand[card]:
                    if (elem - self.hand[card][0]) % 10 == 0:
                        self.stacks[stacklist[stack]].insert(0, elem)
                        card_counter += 1

            self.hand.remove(self.hand[card])
        
        return card_counter, rf"New stacks:{self.stacks}, New hand: {self.hand}"
    
    def nested_list_size(self, hand):
        '''
        Function that computes the length of nested list when flattened.
        Output:
            card_counter (flattened list length)
        '''
        card_counter = 0
        for item in hand: # distinguish between nested items and integers
            if type(item) == list:
                card_counter += len(item)
            else:
                card_counter += 1
        return card_counter
        
    def draw_from_pile(self):
        '''
        Function that draws from the pile, and sorts new cards in hand ignoring combo moves that cannot be sorted.
        '''
        if len(self.drawing_pile) > 0:
            new_card = self.drawing_pile[0]
            print(rf"Newly drawn card: {new_card}")
            self.drawing_pile.remove(new_card) # Remove card from the stack
            self.hand.append(new_card) # and add it to hand

            # sorting the individual cards
            single_cards = [elem for elem in self.hand if isinstance(elem, int)]
            single_cards.sort()
            stacked_cards = [elem for elem in self.hand if isinstance(elem, list)]

            # adding the individual cards and pre-existing ten_diffs back into the hand
            self.hand = single_cards + stacked_cards

    def hand_flattener(self):
        '''
        Function that flattens nested lists in hand to compute the amount of cards that must be drawn from the pile.
        Output:
            - flattened_list (hand with combo moves reduced back to original state)
        '''
        flattened_list = []
        for item in self.hand:
            if type(item) == list:
                for elem in item:
                    flattened_list.append(elem)
            else:
                flattened_list.append(item)
        sorted(flattened_list)
        return flattened_list
    
    def game_end_check(self, output):
        '''
        Function that checks if some output corresponds to an end-of-game message
        Input:
            -output: some function output
        Output:
            -output: if the output corresponds to an end-of-game message, propagate that message.
        '''
        if output in [self.game_over, self.complete]: # check both end-of-game messages
            return(output)

    def full_turn(self, max_extra_move=3):
        '''
        Function that performs a full turn based on the status of the stacks, hand and drawing pile.
        Input:
            max_extra_move (The amount of damage that may be done with an additional card before drawing new ones, defaulted at 3)
        '''
        print(rf"Starting hand and stacks: {self.hand, self.stacks}") # First print current status pre-move
        
        # If the drawing pile is not empty yet, the players must play at least two cards
        if len(self.drawing_pile) >= 1: 
            move = self.play_move() # Play first move

            # If the game is ended after this turn, propagate this message and stop playing cards
            if self.game_end_check(move) != None: 
                return rf"{self.game_end_check(move)} {len(self.hand) + len(self.drawing_pile)} cards left."
            
            # If the first move played was an individual card, play at least one more move
            if self.nested_list_size(self.hand) > 6:
                move = self.play_move()
                if self.game_end_check(move) != None:
                    return rf"{self.game_end_check(move)} {len(self.hand) + len(self.drawing_pile)} cards left."
        
        # If the drawing pile is empty, playing one card suffices.
        else:
            move = self.play_move()
            if self.game_end_check(move) != None:
                return rf"{self.game_end_check(move)} {len(self.hand) + len(self.drawing_pile)} cards left."

        # As long as a next move does not do more damage than the input value, it may be played.
        while self.analyze_move().min() <= max_extra_move:
            move = self.play_move()
            self.game_end_check(move)

        # Compute the amount of cards played and how much needs to be drawn from the pile if possible
        self.hand = self.hand_flattener()
        for _ in range(8 - len(self.hand)): # Fill up hand to 8 cards
            self.draw_from_pile()
        self.hand = self.ten_moves()

        print(rf"Current stacks: {self.stacks}")
        print(rf"Drawing pile size: {len(self.drawing_pile)}")
        print(rf"New hand: {self.hand}")

game_instance = TheGame()

In [48]:
game_instance.full_turn()

Starting hand and stacks: ([3, 31, 32, 33, 53, 55, 62, 69], {'up1': [88, 98, 89, 97, 86, 96, 94, 81, 79, 64, 46, 56, 52, 45, 41, 51, 48, 28, 22, 19, 15, 8, 1], 'up2': [85, 73, 83, 72, 82, 77, 68, 78, 74, 60, 44, 43, 42, 35, 24, 21, 20, 18, 17, 4, 1], 'down1': [2, 5, 6, 7, 9, 10, 11, 36, 37, 27, 34, 54, 57, 59, 63, 67, 75, 65, 84, 90, 100], 'down2': [14, 26, 29, 38, 39, 58, 61, 70, 76, 80, 91, 100]})
11.0
Best card to play: 3 on stack 4.
inf


'No move possible anymore. 24 cards left.'